# Converting sampling result to MIDI data

In [1]:
import numpy as np
import pretty_midi
import pandas as pd
import collections
import glob
import os
from tqdm import tqdm

In [2]:
def convert_to_midi(vector, note_split):
    """
    params
    -------------
    vector: numpy.ndarray
        convert data(channel, length, 88)
    note_split: int
        split number of note
    
    returns
    ---------
    piano: object
        midi_instrument data
    """
    
    shapes = vector.shape
    instrument_name = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=instrument_name)

    #parameter
    start_time = 0.0
    end_time = 0.0
    tempo = 120
    time_split = 60/(tempo*note_split/4)

    #add note
    for p in range(shapes[-1]):
        start_time = 0.0
        end_time = 0.0
        durting = False #judge pitch durting
        on = 0 #judge pitch start
        for t in range(shapes[-2]):
            if not durting:
                on = int(round(vector[0][t][p]))
            end_time += time_split
    
            #check pitch durting on next time
            if on > 0:
                if t != shapes[-2]-1:
                    sutain = int(round(vector[1][t+1][p]))
                    if sutain > 0:
                        durting = True
                        continue
                    else:
                        durting = False
            else:
                start_time = end_time
                durting = False
                continue      
    
            #add note
            note = pretty_midi.Note(
                velocity=70,
                pitch=int(p+21),
                start=start_time, 
                end=end_time
            )
            piano.notes.append(note)
            start_time = end_time
    
    return piano

In [13]:
#file names
file_name = "result/sample_2600.npy"
ground_truth_path = "data/npy/3genre_16note_8bar/test"
result_dir = f"result_midi/sampling" 
#parammeter
use_testdata = False
data_num = 40 
note_split = 16 #varity of split note
observation_names = ['right', "left"] #melody modalities
text_test = [] #genre in test data
text_sample = [] #genre in sample data

#load file
file_name_test = sorted(glob.glob(f"{ground_truth_path}/*.npy"))
os.makedirs(result_dir, exist_ok=True)
vectors_test = [[]for i in range(len(observation_names))]
vectors_sample = [[]for i in range(len(observation_names))]
if use_testdata:
    data_num = len(file_name_test)
    for title in file_name_test:
        vector_result = np.load(title, allow_pickle=True).item()
        for n, key in enumerate(observation_names):
            vector_tmp = vector_result[key]
            vectors_test[n].append(vector_tmp)
        text_test.append(vector_result["text"])

        
vector_result = np.load(file_name, allow_pickle=True).item()
for i in range(data_num):
    for n, key in enumerate(observation_names):
        vector_tmp = vector_result[key][i].to('cpu').detach().numpy().copy()
        vectors_sample[n].append(vector_tmp)
    text_sample.append(vector_result["text"][i])
        
#convert to midi
for i in tqdm(range(data_num)):
    midi = pretty_midi.PrettyMIDI()
    instrument = []
    #input test data
    if use_testdata:
        for n in range(len(observation_names)):
            instrument.append(convert_to_midi(vectors_test[n][i], note_split))
    for n in range(len(observation_names)):
        instrument.append(convert_to_midi(vectors_sample[n][i], note_split))
    
    #add instrument
    if use_testdata:
        for n in range(len(observation_names)*2):
            midi.instruments.append(instrument[n])
    else:
        for n in range(len(observation_names)):
            midi.instruments.append(instrument[n])
    
    #wirte
    wirte_title = os.path.join(result_dir, str(i) + ".mid")
    midi.write(wirte_title)

100%|██████████| 40/40 [00:02<00:00, 13.96it/s]


In [15]:
#display genre condition
for idx, text in enumerate(text_sample):
    print(f"{idx}.mid: {text}")
    

0.mid: jazz song
1.mid: jazz song
2.mid: folk song
3.mid: jazz song
4.mid: classical song
5.mid: jazz song
6.mid: classical song
7.mid: classical song
8.mid: classical song
9.mid: classical song
10.mid: classical song
11.mid: jazz song
12.mid: jazz song
13.mid: jazz song
14.mid: folk song
15.mid: folk song
16.mid: jazz song
17.mid: folk song
18.mid: jazz song
19.mid: jazz song
20.mid: folk song
21.mid: jazz song
22.mid: jazz song
23.mid: folk song
24.mid: jazz song
25.mid: jazz song
26.mid: classical song
27.mid: jazz song
28.mid: folk song
29.mid: classical song
30.mid: folk song
31.mid: classical song
32.mid: jazz song
33.mid: classical song
34.mid: classical song
35.mid: classical song
36.mid: folk song
37.mid: jazz song
38.mid: folk song
39.mid: classical song
